There are several interesting insights obtained from the data exploration. The insights are listed below:
1. The ratio of claim occurence since the first incident claim is doubled in Minneapolis/ St.Paul International Airport compared to other airports. There doesn't seem to be any particular reasoning for this to happen. However, half of the claims occur from passengers using Northwest Airlines. When observed further Northwest Airline isn't the reason for claims in other airports beside of Minneapolis/ St.Paul International Airport and Detroit Airport.

2. There is a slight decreasing trend between how quick a claim is processed (Date Recieved-Incident Date) over the years. 

3. Status was recorded for the very first time for an incident date in 2010. This shown by status' value which is recorded in a different format since 2010.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from ipywidgets import interact

In [2]:
pd.options.display.max_rows = 9999

In [3]:
tsa = pd.read_csv('tsa_claims_ujian.csv')

In [4]:
def info_df(data):
    max_val = []
    min_val = []
    for col in data:
        if col in data.select_dtypes(['datetime','number']).columns:
            max_val.append(data[col].max())
            min_val.append(data[col].min())
        else:
            max_val.append('-')
            min_val.append('-')
            
            
    return pd.DataFrame(dict(dataFeatures=data.columns,
                             dataType=data.dtypes,
                             notNull=data.notnull().sum(),
                             null=data.isna().sum(),
                             nullPct=round(data.isna().sum()/len(data)*100,2),
                             unique=data.nunique(),
                             min_value=min_val,
                             max_value=max_val,
                             uniqueSample=[list(data[i].drop_duplicates().sample(2)) for i in data.columns]
                            )).reset_index(drop=True)

info_df(tsa)

,dataFeatures,dataType,notNull,null,nullPct,unique,min_value,max_value,uniqueSample
0,Claim Number,object,51066,0,0.00,51065,-,-,"[2015042822164, 2010102577320]"
1,Date Received,object,50996,70,0.14,3522,-,-,"[4-Nov-02, 27-Jul-14]"
2,Incident Date,object,50496,570,1.12,10788,-,-,"[11/10/2010 14:28, 6/27/2013 2:05]"
3,Airport Code,object,49014,2052,4.02,405,-,-,"[SBA, HKY]"
4,Airport Name,object,49014,2052,4.02,419,-,-,"[Gustavus, Lihue Airport]"
5,Airline Name,object,42511,8555,16.75,243,-,-,"[China Southern Airlines, Pacific Southwest Ai..."
6,Claim Type,object,49106,1960,3.84,8,-,-,"[Personal Injury, -]"
7,Claim Site,object,50876,190,0.37,6,-,-,"[Motor Vehicle, -]"
8,Item,object,50054,1012,1.98,1854,-,-,[Furniture (including brass & cement furniture...
9,Claim Amount,object,50074,992,1.94,13248,-,-,"[$539.00 , $194.98 ]"


In [5]:
#replace '-'
tsa.replace({'-':np.nan},inplace=True)

#change Date Received and Incident Date to Datetime
def year_cleaning(c):
    try:
        pd.to_datetime(c[0])
        return c[0]
    except:
        return re.sub(r'\d{4}',str(pd.to_datetime(c[1]).year),c[0])

tsa['Incident Date'] = tsa[['Incident Date','Date Received']].apply(year_cleaning,axis=1)

for i in ['Date Received','Incident Date']:
    tsa[i] = pd.to_datetime(tsa[i])
    
#add Day Difference feature
tsa['Day Difference'] = (tsa['Date Received']-tsa['Incident Date']).dt.days

#Claim Amount and Close Amount cleaning
for i in ['Claim Amount','Close Amount']:
    tsa[i] = tsa[i].apply(lambda c: float(re.sub(r'([\D]{1}(?=.\d{2})|\$)','',c) if not pd.isna(c) else c))
    
#add Amount Difference feature
tsa['Amount Differences'] = tsa['Claim Amount']-tsa['Close Amount']

tsa_null = tsa.copy()

In [6]:
info_df(tsa)

,dataFeatures,dataType,notNull,null,nullPct,unique,min_value,max_value,uniqueSample
0,Claim Number,object,51066,0,0.00,51065,-,-,"[2011122089783, 2004063057973]"
1,Date Received,datetime64[ns],50996,70,0.14,3522,2000-09-08 00:00:00,2055-05-17 00:00:00,"[2014-12-16 00:00:00, 2009-02-17 00:00:00]"
2,Incident Date,datetime64[ns],50496,570,1.12,10641,2000-01-01 00:00:00,2015-12-29 00:00:00,"[2012-12-06 05:15:00, 2011-11-19 00:00:00]"
3,Airport Code,object,48915,2151,4.21,404,-,-,"[MAF, JFK]"
4,Airport Name,object,48915,2151,4.21,418,-,-,"[University of Illinois-Willard, Waterloo Muni..."
5,Airline Name,object,41470,9596,18.79,242,-,-,"[Southwest Airlines , Arika Isreali Airlines ]"
6,Claim Type,object,49035,2031,3.98,7,-,-,"[Employee Loss (MPCECA), Complaint]"
7,Claim Site,object,50817,249,0.49,5,-,-,"[Checkpoint, Checked Baggage]"
8,Item,object,48918,2148,4.21,1853,-,-,[Cosmetics - Perfume; toilet articles; medicin...
9,Claim Amount,float64,46886,4180,8.19,13247,0,1.25e+08,"[224.86, 92.75]"


In [7]:
nan_fill = {}

types = ['object','datetime','number']
fills = ['null',pd.Timestamp(1900,12,12),-3000]
for typ, fill in zip(types,fills):
    for col in [cols for cols in tsa.select_dtypes(typ)]:
        nan_fill[col] = fill

# nan_fill
tsa.fillna(nan_fill,inplace=True)

In [ ]:
def info_filled_df(data):
    max_val = []
    min_val = []
    for col,nan in nan_fill.items():
        if col in data.select_dtypes(['datetime','number']).columns:
            max_val.append(data[col].replace({nan:np.nan}).dropna().max())
            min_val.append(data[col].replace({nan:np.nan}).dropna().min())
        else:
            max_val.append('-')
            min_val.append('-')
            
            
    return pd.DataFrame(dict(dataFeatures=data.columns,
                             dataType=data.dtypes,
                             notNull=data.notnull().sum(),
                             null=data.isna().sum(),
                             nullPct=round(data.isna().sum()/len(data)*100,2),
                             nullFill=[i for i in nan_fill.values()],
                             unique=data.nunique(),
                             min_value=min_val,
                             max_value=max_val,
                             uniqueSample=[list(data[i].drop_duplicates().sample(2)) for i in data.columns]
                            )).reset_index(drop=True)

info_filled_df(tsa)

,dataFeatures,dataType,notNull,null,nullPct,nullFill,unique,min_value,max_value,uniqueSample
0,Claim Number,object,51066,0,0.0,null,51065,-,-,"[2004090362118, 2007091832139]"
1,Date Received,datetime64[ns],51066,0,0.0,null,3523,-,-,"[2003-03-07 00:00:00, 2007-08-16 00:00:00]"
2,Incident Date,datetime64[ns],51066,0,0.0,null,10642,-,-,"[2009-10-17 21:15:00, 2012-01-02 00:00:00]"
3,Airport Code,object,51066,0,0.0,null,405,-,-,"[YKM, DSM]"
4,Airport Name,object,51066,0,0.0,null,419,-,-,"[Lake Charles Regional Airport, Hays Regional]"
5,Airline Name,object,51066,0,0.0,null,243,-,-,"[Air China, Mesa Airlines]"
6,Claim Type,object,51066,0,0.0,null,8,-,-,"[Employee Loss (MPCECA), Passenger Theft]"
7,Claim Site,object,51066,0,0.0,null,6,-,-,"[Checkpoint, Other]"
8,Item,object,51066,0,0.0,null,1854,-,-,[Cameras - Digital; Cameras - Non-electronic (...
9,Claim Amount,float64,51066,0,0.0,1900-12-12 00:00:00,13248,2000-09-08 00:00:00,2055-05-17 00:00:00,"[162.6, 21.45]"


In [ ]:
tsa.drop(columns='Claim Number').describe(include='all').transpose()

In [ ]:
object_cols = [i for i in tsa.select_dtypes('object') if i != 'Claim Number']
datetime_cols = [i for i in tsa.select_dtypes('datetime')]
number_cols = [i for i in tsa.select_dtypes('number')]

In [ ]:
for i in datetime_cols:
    tsa[f'day_{i}'] = tsa[i].apply(lambda x: x.day_name() if x!=fills[1] else 'null')
    tsa[f'month_{i}'] = tsa[i].apply(lambda x: x.month_name() if x!=fills[1] else 'null')
    tsa[f'year_{i}'] = tsa[i].apply(lambda x: x.year if x!=fills[1] else -3000)

tsa.head()

In [ ]:
def Airport_Code_Name(c='Airport Code',n='Airport Name'):
    code_check = {i:[] for i in tsa[c]}
    name_check = {i:[] for i in tsa[n]}
    for code, name in zip(tsa[c],tsa[n]):
        code_check[code].append(name)
        name_check[name].append(code)

    for dic,s in zip([code_check,name_check],[c,n]):
        print(f'\nDuplicates in {s}\n')
        dupes = []
        for key,value in dic.items():
            value_set = set(value)
            if len(value_set)!=1:
                dupes.append(value)
                print(f'key = {key}')
                print(f'duplicate values = {value_set}')
        if len(dupes)==0:
            print("There are no duplicates")
                
Airport_Code_Name()

In [ ]:
tsa['Airport Name'] = tsa['Airport Name'].apply(lambda x: x.strip() if not pd.isna(x) else x)
Airport_Code_Name()

In [ ]:
PNS_Airport_describe = tsa[tsa['Airport Code']=='PNS'].groupby(
    'Airport Name').describe(
    include='all').drop(
    columns='Claim Number').transpose()

PNS_Airport_describe

In [ ]:
idx = [i for i in PNS_Airport_describe]
val = {'ratio':[],'claim_quantity':[],'days_since_first_claim':[]}
for i in PNS_Airport_describe:
    first = PNS_Airport_describe.loc[('Incident Date','first'),i]
    last_regional = PNS_Airport_describe.loc[('Incident Date','last'),i]
    if 'Regional' in i:
        duration = (last_regional-first).days
    else:
        duration = (tsa['Incident Date'].describe()['last']-first).days
    count=PNS_Airport_describe.loc[('Incident Date','count'),i]
    ratio=count/duration*100
    for v,value in zip(val.keys(),[ratio,count,duration]):
        val[v].append(value)
pd.DataFrame(val,idx)    

In [ ]:
airport_code_grp = tsa[['Airport Code','Incident Date']].groupby('Airport Code').describe()
last_claim = tsa['Incident Date'].describe()['last']
idx = []
val = {'ratio':[],'claim_quantity':[],'days_since_first_claim':[]}
for i in airport_code_grp.index:
    idx.append(i)
    first_claim = airport_code_grp.loc[i,('Incident Date','first')]
#     last = airport_code_grp.loc[i,('Incident Date','last')]
    duration = (last_claim-first_claim).days
    count = airport_code_grp.loc[i,('Incident Date','count')]
#     if duration==0:
#         ratio = -count
#     else:
    ratio = count/duration*100
    for v,value in zip(val.keys(),[ratio,count,duration]):
        val[v].append(value)
df_ratio = pd.DataFrame(val,idx).sort_values('ratio',ascending=False)

In [ ]:
tsa[tsa['Airport Code']=='MSP'].describe(include='all').drop(columns=['Claim Number','Airport Name','Airport Code']).transpose()

In [ ]:
msp = 'Minneapolis/ St.Paul International Airport'
tsa['msp']=tsa['Airport Code'].apply(lambda c: msp if c=='MSP' else f'Not {msp}')

In [ ]:
x,y = 'Date Received','Incident Date'
figure = go.Figure({'data':[go.Scatter(
                x= tsa[tsa['msp']==i][x],
                y= tsa[tsa['msp']==i][y],
                mode='markers',
                name= f'{i}',    
            ) for i in tsa['msp'].unique()
            ],
                'layout':go.Layout(
                    xaxis= {'title':x},
                    yaxis = {'title':y},
                    hovermode = 'closest',
                    title = f'{x} vs {y}'
                )
            })

iplot(figure)

In [ ]:
sns.set_style('whitegrid')
sns.scatterplot('Claim Amount','Close Amount',data=tsa[tsa['Airport Code']=='MSP'])

In [ ]:
objcts = [i for i in object_cols if not i in ['Airport Name','Airport Code']]
fig,axes = plt.subplots(len(objcts),2,figsize=(16,16*len(objcts)))
for ratio, ax in zip(objcts,axes):
    for filt,a in zip(tsa['msp'].unique(),ax):
        data = tsa[tsa['msp']==filt][ratio].value_counts(normalize=True).head(10)
        a.set_title(f'{ratio} Ratio\n({filt})\n')
        sns.barplot(x=data.index,y=data,ax=a,orient='v')
        a.tick_params('x',labelrotation=90)
plt.tight_layout()

In [ ]:
fig,axes = plt.subplots(4,2,figsize=(16,8*4))
for ratio, ax in zip(number_cols,axes):
    for filt,a in zip(tsa['msp'].unique(),ax):
        data = tsa[tsa['msp']==filt][ratio].value_counts(normalize=True).head(10)
        a.set_title(f'{ratio} Ratio\n({filt})\n')
        sns.barplot(x=data.index,y=data,ax=a)
plt.tight_layout()

In [ ]:
ser = tsa[tsa['Airline Name']=='Northwest Airlines']['Airport Name']
sns.countplot(y=ser,order=ser.value_counts().head(10).index)
plt.title(f'''Total claims from passengers using Northwest Airlines in each Airports
          (There are {len(ser)} claims from {ser.nunique()} airports)''')
for y,x in enumerate(ser.value_counts().head(10)):
    pct = round(x/len(ser)*100,2)
    plt.annotate(f'{x} ({pct}%)',(x+5,(y+.14)))

In [ ]:
ser = tsa[tsa['Airport Name'].str.contains('Detroit')]['Airline Name']
sns.countplot(y=ser,order=ser.value_counts().head(10).index)
plt.title(f'''Total claims from passengers landing in
Detroit Metropolitan Wayne Country Airport from each Airlines
(There are {len(ser)} claims from {ser.nunique()} airlines)''')
for y,x in enumerate(ser.value_counts().head(10)):
    pct = round(x/len(ser)*100,2)
    plt.annotate(f'{x} ({pct}%)',(x+5,(y+.14)))

In [ ]:
northwest = tsa[tsa['Airline Name']=='Northwest Airlines']
airports = northwest['Airport Name'].unique()
# airports = [i for i in northwest['Airport Name'].unique() if len(tsa[tsa['Airport Name']==i])>100]
# fig,ax = plt.subplots(len(airports),1,figsize=(12,len(airports)*7))
# for ap,a in zip(airports,ax):
#     ser=tsa[tsa['Airport Name']==ap]['Airline Name']
#     sns.countplot(y=ser,order=ser.value_counts().head(10).index,ax=a)
#     a.set_title(f'''Total claims from passengers landing in {ap}
#     (There are {len(ser)} claims)''')
#     for y,x in enumerate(ser.value_counts().head(10)):
#         pct = round(x/len(ser)*100,2)
#         a.annotate(f'{x} ({pct}%)',(x*1.1,(y+.14)))
# plt.tight_layout()

index=[]
columns = ['count_all','count_northwest','percentage']
count={f'{i}':[] for i in columns}

for idx in airports:
    index.append(idx)
    al = len(tsa[tsa['Airport Name']==idx])
    nwa = len(northwest[northwest['Airport Name']==idx])
    perc = nwa/al*100
    for key, val in zip(columns,[al,nwa,perc]):
        count[key].append(val)

pd.DataFrame(count,index).sort_values(['count_northwest','percentage'],ascending=False)

In [ ]:
cutoff = tsa['Airport Code'].value_counts().head(10).index
sns.scatterplot('Airport Code','Airline Name',data=tsa[tsa['Airport Code'].isin(cutoff)])

In [ ]:
cutoff = tsa['Airline Name'].value_counts().head(10).index
sns.scatterplot('Airport Code','Airline Name',data=tsa[tsa['Airline Name'].isin(cutoff)])

In [ ]:
df_count = pd.DataFrame({'claims count':tsa.groupby(['Airport Name','Airline Name']).count()['Claim Number']})
df_count['claims percentage'] = df_count['claims count'].apply(lambda c:round(c/len(tsa)*100,2))
df_count.sort_values(['claims percentage','claims count'],ascending=False)

In [ ]:
sns.heatmap(tsa.select_dtypes('number').corr(),annot=True)

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(14,8*2))
# sns.distplot(tsa['year_Incident Date'])
# sns.distplot(tsa['Day Difference'])
sns.scatterplot('year_Incident Date','Day Difference',data=tsa[tsa['year_Incident Date']!=-3000],ax=axes[0])
axes[0].set_title('Incident Date Year != null')
sns.scatterplot('year_Incident Date','Day Difference',data=tsa[tsa['year_Incident Date']==-3000],ax=axes[1])
axes[1].set_title('Incident Date Year == null')

In [ ]:
tsa.groupby('year_Incident Date').mean()['Day Difference']

In [ ]:
tsa[tsa['Day Difference']>360].describe(include='all').drop(columns=['Claim Number','msp']).transpose()

In [ ]:
tsa[tsa['Day Difference']<0].describe(include='all').drop(columns=['Claim Number','msp']).transpose()

In [ ]:
tsa[tsa['Date Received']>tsa['Incident Date'].max()].describe(include='all').drop(columns=['Claim Number','msp']).transpose()

In [ ]:
sns.scatterplot('Day Difference','Amount Differences',data=tsa[(tsa['Amount Differences']!=-3000)&(tsa['Day Difference']!=-3000)])

In [ ]:
tsa[(tsa['Claim Amount']==-3000)&(tsa['Close Amount']==-3000)].describe(include='all').drop(columns=['Claim Number','msp']).transpose()

In [ ]:
tsa_null[tsa_null['Disposition'].isna()].describe(include='all').drop(columns='Claim Number').transpose()

In [ ]:
tsa_null[tsa_null['Status'].isna()].describe(include='all').drop(columns='Claim Number').transpose()

In [ ]:
def sd(cols):
    status=cols[0]
    disp=cols[1]
    if pd.isna(status) and pd.isna(disp):
        return 'both null'
    elif not(pd.isna(status) and pd.isna(disp)):
        return 'none null'
    elif pd.isna(status) and not pd.isna(disp):
        return 'status null'
    else:
        return 'disposition null'
tsa_null['status_disposition'] = tsa_null[['Status','Disposition']].apply(sd,axis = 1)

In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(y='Incident Date',x='Status',hue='status_disposition',data=tsa_null)
plt.xticks(rotation=90)


In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(y='Incident Date',x='Disposition',hue='status_disposition',data=tsa_null)
# plt.xticks(rotation=90)


In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(y='Disposition',x='Status',hue='year_Incident Date',data=tsa[tsa['year_Incident Date']!=-3000],palette='plasma')
plt.xticks(rotation=90)


In [ ]:
tsa[tsa['Disposition']=='null']['Status'].value_counts()

In [ ]:
# plt.figure(figsize=(20,4))
# sns.countplot(y='Airport Code',data=tsa,order=tsa['Airport Code'].value_counts().head(10).index)
# sns.despine(bottom=True)
# plt.xticks([])
# plt.xlabel('')
# top10 = []
# for y,x in enumerate(tsa['Airport Code'].value_counts().head(10)):
#     pct = round(x/len(tsa)*100,2)
#     plt.annotate(f'{x} ({pct}%)',(x+10,(y+.14)))
#     top10.append(pct)
# top10 = round(sum(top10),2)
# plt.title(f'Countplot of Airport Code\n(Top 10, {top10}% of the data)\n')

In [ ]:
# tsa.drop(columns='Claim Number')[tsa['Airport Code']=='null'].describe(include='all')

In [ ]:
# fig, axes = plt.subplots(len(object_cols),1,figsize=(12,12*len(object_cols)))

# for col, ax in zip(object_cols,axes):
#     ser = tsa[col]
#     sns.countplot(y=ser,ax=ax,order=ser.value_counts().head(10).index)
